- 先觀察實際下載所用的api請求後，可以發現主要是透過`https://www.stat.gov.tw/CCI/CCI_Site/CCIPrice/PartialPriceClassesList.aspx` 取得資料，所以先用postman測試看看能不能不透過UI的方式來進行資料的取得

In [1]:
import requests

In [2]:
url = "https://www.stat.gov.tw/CCI/CCI_Site/CCIPrice/PartialPriceClassesList.aspx"

payload = "{\"cmd\":\"DwExL\",\"data\":\"{\\\"CategoriesId\\\":\\\"11,37,38,58,62,72,99,103\\\",\\\"CategoryIds\\\":\\\"8\\\",\\\"CategoryType\\\":\\\"PCLS\\\",\\\"CorrespondsBYr\\\":2013,\\\"CorrespondsBMn\\\":1,\\\"CorrespondsEYr\\\":2024,\\\"CorrespondsEMn\\\":5}\"}"
headers = {
  'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Mobile Safari/537.36',
  'Accept': 'application/json',
  'Connection': 'keep-alive',
  'host': 'www.stat.gov.tw',
  'Content-Length': str(len(payload)),
  'Content-Type': 'text/plain'
}

response = requests.request("POST", url, headers=headers, data=payload)
print(response.text)

{"TotalItems":0,"ItemsPerPage":0,"CurrentPage":0,"TotalPages":0,"PageHtml":"<table border=1 ><tr><td width='70' class='title'>年/月</td><td width='60' valign='top'>不含11.瓦片及紅磚之[磚瓦瓷類]指數</td><td width='60' valign='top'>不含37.木門、38.木板之[木材及其製品類]指數</td><td width='60' valign='top'>不含58.泵浦、62.感知器之[機電設備類]指數</td><td width='60' valign='top'>不含72.玻璃製品之[雜項類]指數</td><td width='60' valign='top'>不含99.挖土機租金、103.舖面機租金之[機具設備租金類]指數</td><td width='60' valign='top'>不含11.瓦片及紅磚、37.木門、38.木板、58.泵浦、62.感知器、72.玻璃製品、99.挖土機租金、103.舖面機租金、[砂石及級配類]之總指數</td><td width='60' valign='top'>11.瓦片及紅磚指數</td><td width='60' valign='top'>37.木門指數</td><td width='60' valign='top'>38.木板指數</td><td width='60' valign='top'>58.泵浦指數</td><td width='60' valign='top'>62.感知器指數</td><td width='60' valign='top'>72.玻璃製品指數</td><td width='60' valign='top'>99.挖土機租金指數</td><td width='60' valign='top'>103.舖面機租金指數</td><td width='60' valign='top'>砂石及級配類指數</td></tr><tr><td>102/01</td><td>97.00</td><td>81.40</td><td>89.35</td><td>89.26</td><td>89.89</td><td>84.2

- 既然確定這個方式可以取得資料，剩下就是把資料湊成csv的格式進行儲存了

In [3]:
cells = response.text.split('<tr>')
cells.pop(0)

'{"TotalItems":0,"ItemsPerPage":0,"CurrentPage":0,"TotalPages":0,"PageHtml":"<table border=1 >'

In [4]:
data = []
for i in range(len(cells)):
    c = []
    if i == 0: 
        title = cells[0].replace('<td>','').replace('</tr>','').split('</td>')
        title.pop()
        for t in title:
            c.append(t.split('>')[1])
    elif i != len(cells)-1: 
        c = cells[i].replace('<td>','').replace('</tr>','').split('</td>')
        c.pop()
    else:
        c = cells[i].split('</tr>')[0].replace('<td>','').replace('</tr>','').split('</td>')
        c.pop()
    data.append(','.join(c))

In [5]:
file = 'data.csv'
with open(file, 'w', encoding='cp950') as f:
    f.write('\n'.join(data))